In [1]:
import sys
import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder

def SMAPE(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

In [12]:
train_day = pd.read_csv('../train.csv')
train_day

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


In [13]:
train = pd.read_csv('final_train.csv')
test = pd.read_csv('final_test.csv')
#train_day = pd.read_csv('../train.csv')
#test_day = pd.read_csv('../test.csv')
#train = pd.read_csv('')
train

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),태양광용량(kW),hour,day,month,week,...,type_day_hour_std,type_hour_mean,type_hour_std,holiday,sin_time,cos_time,THI,CDH,7_shifted_전력소비량,전력소비량(kWh)
0,1,19.0,1.889045,3.2,61.0,0,0,2,6,23,...,886.546805,1616.129012,909.159339,0,0.000000,1.000000,48.2249,-38.9,1085.28,1124.16
1,1,18.8,1.889045,2.6,61.0,0,1,2,6,23,...,906.029700,1603.843635,921.827876,0,0.258819,0.965926,47.8649,-45.1,1047.36,1059.36
2,1,18.5,1.889045,2.6,62.0,0,2,2,6,23,...,900.729730,1575.108376,916.043890,0,0.500000,0.866025,47.4096,-51.4,974.88,987.36
3,1,18.1,1.889045,2.2,63.0,0,3,2,6,23,...,900.693834,1553.335094,908.635022,0,0.707107,0.707107,46.7941,-58.0,953.76,977.76
4,1,17.2,1.889045,3.2,66.0,0,4,2,6,23,...,885.287208,1543.278965,900.520543,0,0.866025,0.500000,45.6064,-64.9,986.40,1009.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187195,100,23.1,2.533115,0.9,86.0,0,19,2,8,34,...,812.561591,1728.999353,821.346561,0,-0.965926,0.258819,63.6624,-19.6,1049.52,881.04
187196,100,22.4,2.533115,1.3,86.0,0,20,2,8,34,...,784.696267,1648.600324,786.021787,0,-0.866025,0.500000,62.4024,-20.2,874.32,798.96
187197,100,21.3,2.533115,1.0,92.0,0,21,2,8,34,...,754.000774,1506.193235,757.941653,0,-0.707107,0.707107,64.1976,-22.3,678.24,825.12
187198,100,21.0,2.533115,0.3,94.0,0,22,2,8,34,...,644.114833,1263.618397,643.779849,0,-0.500000,0.866025,65.0744,-25.1,632.64,640.08


In [9]:
train.columns

Index(['건물번호', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '태양광용량(kW)', 'hour',
       'day', 'month', 'week', 'month_day_mean', 'day_hour_mean',
       'day_hour_std', 'hour_mean', 'hour_std', 'type_day_hour_mean',
       'type_day_hour_std', 'type_hour_mean', 'type_hour_std', 'holiday',
       'sin_time', 'cos_time', 'THI', 'CDH', '7_shifted_전력소비량', '전력소비량(kWh)'],
      dtype='object')

## XGB_ALL

In [10]:
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

In [14]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def objective_xgb(trial: Trial, X_train, y_train, X_val, y_val):
    params = {
        "n_estimators": trial.suggest_int('n_estimators', 500, 5000),
        'max_depth': trial.suggest_int('max_depth', 8, 16),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'gamma': trial.suggest_int('gamma', 1, 3),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        #'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        #'alpha': trial.suggest_loguniform('alpha', 1e-3, 100.0),
        #'alpha': trial.suggest_loguniform('alpha', 1, 100.0),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1, 100.0, log=True),
        'subsample': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 1.0]),
        'random_state': 724
    }

    model = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0, seed=724, verbosity=2)
    alpha_value = model.get_params()['alpha']
    model.set_params(**{'objective':weighted_mse(alpha_value)})
    model.fit(X_train, y_train, verbose = False, eval_set=[(X_val, y_val)], early_stopping_rounds=50)
    y_pred = model.predict(X_val)
    score = SMAPE(y_val, y_pred)

    return score


In [16]:
# 0
preds = np.array([])
best_params = {}
best_values = {}  # New dictionary to store the best values
with open('best_params_values_0825_0.txt', 'w') as f:
    # Select the data for the current num
    num_data = train.copy()

    # Combine 'hour', 'day', 'week' to a new feature as stratified target
    #num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
    num_data['stratified_target'] = num_data['건물번호'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['hour'].astype(str)

    # Split the data into training and validation set
    train_df, val_df = train_test_split(num_data, test_size=0.7, stratify=num_data['stratified_target'], random_state=724)

    # Drop the temporary feature
    train_df = train_df.drop(columns='stratified_target')
    val_df = val_df.drop(columns='stratified_target')

    # Validation set에서 day 값이 0이 아닌 행을 training set에 추가
    train_df = pd.concat([train_df, val_df[~val_df['day'].isin([0])]])

    # Validation set에서 day 값이 0인 행만 남기기
    val_df = val_df[val_df['day'].isin([0])]

    y_train = train_df['전력소비량(kWh)']
    y_val = val_df['전력소비량(kWh)']

    x_train, x_test = train_df.copy(), test.copy()
    x_val = val_df.copy()

    x_train.drop(['전력소비량(kWh)'],axis=1,inplace=True)
    x_val.drop(['전력소비량(kWh)'],axis=1,inplace=True)

    x_test = x_test[x_train.columns]
    print(len(x_train), len(x_val), x_val['day'].unique(), x_val['건물번호'].nunique(), x_val['hour'].nunique())
    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=15)
    param = study.best_trial.params
    best_params = param
    best_values = study.best_trial.value  # Store the best value
    f.write(f'Best Params: {best_params}, \nBest Values: {best_values}\n\n')
    f.flush()
    xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=953)
    alpha_value2 = xgb.get_params()['alpha']
    xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    pred_0 = np.append(preds, y_pred)


[I 2023-08-26 20:00:54,478] A new study created in memory with name: no-name-0553c870-62f7-4f99-82c7-6fa363daf42c


168538 18662 [0] 100 24


c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-08-26 20:01:29,675] Trial 0 finished with value: 6.83630817487441 and parameters: {'n_estimators': 1966, 'max_depth': 12, 'min_child_weight': 245, 'gamma': 3, 'learning_rate': 0.014, 'colsample_bytree': 0.6430613407852359, 'lambda': 0.0778992548994197, 'alpha': 3.870905691174313, 'subsample': 0.6}. Best is trial 0 with value: 6.83630817487441.
c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-08-26 20:01:54,246] Trial 1 finished with value: 7.407836531983126 and parameters: {'n_estimator

In [15]:
# 1
preds = np.array([])
best_params = {}
best_values = {}  # New dictionary to store the best values
with open('best_params_values_0825_1.txt', 'w') as f:
    # Select the data for the current num
    num_data = train.copy()

    # Combine 'hour', 'day', 'week' to a new feature as stratified target
    #num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
    num_data['stratified_target'] = num_data['건물번호'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['hour'].astype(str)

    # Split the data into training and validation set
    train_df, val_df = train_test_split(num_data, test_size=0.7, stratify=num_data['stratified_target'], random_state=724)

    # Drop the temporary feature
    train_df = train_df.drop(columns='stratified_target')
    val_df = val_df.drop(columns='stratified_target')

    # Validation set에서 day 값이 0이 아닌 행을 training set에 추가
    train_df = pd.concat([train_df, val_df[~val_df['day'].isin([1])]])

    # Validation set에서 day 값이 0인 행만 남기기
    val_df = val_df[val_df['day'].isin([1])]

    y_train = train_df['전력소비량(kWh)']
    y_val = val_df['전력소비량(kWh)']
    
    x_train, x_test = train_df.copy(), test.copy()
    x_val = val_df.copy()

    x_train.drop(['전력소비량(kWh)'],axis=1,inplace=True)
    x_val.drop(['전력소비량(kWh)'],axis=1,inplace=True)

    x_test = x_test[x_train.columns]

    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=15)
    param = study.best_trial.params
    best_params = param
    best_values = study.best_trial.value  # Store the best value
    f.write(f'Best Params: {best_params}, \nBest Values: {best_values}\n\n')
    f.flush()
    xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=953)
    alpha_value2 = xgb.get_params()['alpha']
    xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    pred_1 = np.append(preds, y_pred)


[I 2023-08-25 01:23:19,674] A new study created in memory with name: no-name-fe0a278d-9ae6-4ee0-8c97-459655329667
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-08-25 01:28:34,440] Trial 0 finished with value: 4.255823374085603 and parameters: {'n_estimators': 2736, 'max_depth': 8, 'min_child_weight': 39, 'gamma': 2, 'learning_rate': 0.012, 'colsample_bytree': 0.8179836403047569, 'lambda': 2.6942762098193436, 'alpha': 2.9136439818095887, 'subsample': 0.8}. Best is trial 0 with value: 4.255823374085603.
[I 2023-08-25 01:38:15,107] Trial 1 finished with value: 3.638964764141043 and parameters: {'n_estimators': 4666, 'max_depth': 11, 'min_child_weight': 82, 'gamma': 3, 'learning_rate': 0.01, 'colsample_bytree': 0.7054693692062667, 'lambda': 0.0150191590603315, 'alpha': 1.3

In [ ]:
# 1
preds = np.array([])
best_params = {}
best_values = {}  # New dictionary to store the best values
with open('best_params_values_0825_2.txt', 'w') as f:
    # Select the data for the current num
    num_data = train.copy()

    # Combine 'hour', 'day', 'week' to a new feature as stratified target
    #num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
    num_data['stratified_target'] = num_data['건물번호'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['hour'].astype(str)

    # Split the data into training and validation set
    train_df, val_df = train_test_split(num_data, test_size=0.7, stratify=num_data['stratified_target'], random_state=724)

    # Drop the temporary feature
    train_df = train_df.drop(columns='stratified_target')
    val_df = val_df.drop(columns='stratified_target')

    # Validation set에서 day 값이 0이 아닌 행을 training set에 추가
    train_df = pd.concat([train_df, val_df[~val_df['day'].isin([2])]])

    # Validation set에서 day 값이 0인 행만 남기기
    val_df = val_df[val_df['day'].isin([2])]

    y_train = train_df['전력소비량(kWh)']
    y_val = val_df['전력소비량(kWh)']

    x_train, x_test = train_df.copy(), test.copy()
    x_val = val_df.copy()

    x_train.drop(['전력소비량(kWh)'],axis=1,inplace=True)
    x_val.drop(['전력소비량(kWh)'],axis=1,inplace=True)

    x_test = x_test[x_train.columns]

    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=15)
    param = study.best_trial.params
    best_params = param
    best_values = study.best_trial.value  # Store the best value
    f.write(f'Best Params: {best_params}, \nBest Values: {best_values}\n\n')
    f.flush()
    xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=953)
    alpha_value2 = xgb.get_params()['alpha']
    xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    pred_2 = np.append(preds, y_pred)


[I 2023-08-25 03:00:49,975] A new study created in memory with name: no-name-c359f472-a793-4ff3-a930-9d71811aeacd
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-08-25 03:11:46,633] Trial 0 finished with value: 4.0490465693869595 and parameters: {'n_estimators': 4831, 'max_depth': 15, 'min_child_weight': 123, 'gamma': 3, 'learning_rate': 0.014, 'colsample_bytree': 0.6321523185080865, 'lambda': 5.360325927671868, 'alpha': 1.6902914053128546, 'subsample': 0.6}. Best is trial 0 with value: 4.0490465693869595.
[I 2023-08-25 03:16:57,593] Trial 1 finished with value: 4.488178022066473 and parameters: {'n_estimators': 1125, 'max_depth': 16, 'min_child_weight': 47, 'gamma': 1, 'learning_rate': 0.012, 'colsample_bytree': 0.5435158204677171, 'lambda': 0.008930380038361723, 'alpha

In [ ]:
# 1
preds = np.array([])
best_params = {}
best_values = {}  # New dictionary to store the best values
with open('best_params_values_0825_3.txt', 'w') as f:
    # Select the data for the current num
    num_data = train.copy()

    # Combine 'hour', 'day', 'week' to a new feature as stratified target
    #num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
    num_data['stratified_target'] = num_data['건물번호'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['hour'].astype(str)

    # Split the data into training and validation set
    train_df, val_df = train_test_split(num_data, test_size=0.7, stratify=num_data['stratified_target'], random_state=724)

    # Drop the temporary feature
    train_df = train_df.drop(columns='stratified_target')
    val_df = val_df.drop(columns='stratified_target')

    # Validation set에서 day 값이 0이 아닌 행을 training set에 추가
    train_df = pd.concat([train_df, val_df[~val_df['day'].isin([3])]])

    # Validation set에서 day 값이 0인 행만 남기기
    val_df = val_df[val_df['day'].isin([3])]

    y_train = train_df['전력소비량(kWh)']
    y_val = val_df['전력소비량(kWh)']

    x_train, x_test = train_df.copy(), test.copy()
    x_val = val_df.copy()

    x_train.drop(['전력소비량(kWh)'],axis=1,inplace=True)
    x_val.drop(['전력소비량(kWh)'],axis=1,inplace=True)

    x_test = x_test[x_train.columns]

    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=15)
    param = study.best_trial.params
    best_params = param
    best_values = study.best_trial.value  # Store the best value
    f.write(f'Best Params: {best_params}, \nBest Values: {best_values}\n\n')
    f.flush()
    xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=953)
    alpha_value2 = xgb.get_params()['alpha']
    xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    pred_3 = np.append(preds, y_pred)


In [ ]:
# 1
preds = np.array([])
best_params = {}
best_values = {}  # New dictionary to store the best values
with open('best_params_values_0825_4.txt', 'w') as f:
    # Select the data for the current num
    num_data = train.copy()

    # Combine 'hour', 'day', 'week' to a new feature as stratified target
    #num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
    num_data['stratified_target'] = num_data['건물번호'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['hour'].astype(str)

    # Split the data into training and validation set
    train_df, val_df = train_test_split(num_data, test_size=0.7, stratify=num_data['stratified_target'], random_state=724)

    # Drop the temporary feature
    train_df = train_df.drop(columns='stratified_target')
    val_df = val_df.drop(columns='stratified_target')

    # Validation set에서 day 값이 0이 아닌 행을 training set에 추가
    train_df = pd.concat([train_df, val_df[~val_df['day'].isin([4])]])

    # Validation set에서 day 값이 0인 행만 남기기
    val_df = val_df[val_df['day'].isin([4])]

    y_train = train_df['전력소비량(kWh)']
    y_val = val_df['전력소비량(kWh)']

    x_train, x_test = train_df.copy(), test.copy()
    x_val = val_df.copy()

    x_train.drop(['전력소비량(kWh)'],axis=1,inplace=True)
    x_val.drop(['전력소비량(kWh)'],axis=1,inplace=True)

    x_test = x_test[x_train.columns]

    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=15)
    param = study.best_trial.params
    best_params = param
    best_values = study.best_trial.value  # Store the best value
    f.write(f'Best Params: {best_params}, \nBest Values: {best_values}\n\n')
    f.flush()
    xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=953)
    alpha_value2 = xgb.get_params()['alpha']
    xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    pred_4 = np.append(preds, y_pred)


In [ ]:
# 1
preds = np.array([])
best_params = {}
best_values = {}  # New dictionary to store the best values
with open('best_params_values_0825_5.txt', 'w') as f:
    # Select the data for the current num
    num_data = train.copy()

    # Combine 'hour', 'day', 'week' to a new feature as stratified target
    #num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
    num_data['stratified_target'] = num_data['건물번호'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['hour'].astype(str)

    # Split the data into training and validation set
    train_df, val_df = train_test_split(num_data, test_size=0.7, stratify=num_data['stratified_target'], random_state=724)

    # Drop the temporary feature
    train_df = train_df.drop(columns='stratified_target')
    val_df = val_df.drop(columns='stratified_target')

    # Validation set에서 day 값이 0이 아닌 행을 training set에 추가
    train_df = pd.concat([train_df, val_df[~val_df['day'].isin([5])]])

    # Validation set에서 day 값이 0인 행만 남기기
    val_df = val_df[val_df['day'].isin([5])]

    y_train = train_df['전력소비량(kWh)']
    y_val = val_df['전력소비량(kWh)']

    x_train, x_test = train_df.copy(), test.copy()
    x_val = val_df.copy()

    x_train.drop(['전력소비량(kWh)'],axis=1,inplace=True)
    x_val.drop(['전력소비량(kWh)'],axis=1,inplace=True)

    x_test = x_test[x_train.columns]

    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=15)
    param = study.best_trial.params
    best_params = param
    best_values = study.best_trial.value  # Store the best value
    f.write(f'Best Params: {best_params}, \nBest Values: {best_values}\n\n')
    f.flush()
    xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=953)
    alpha_value2 = xgb.get_params()['alpha']
    xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    pred_5 = np.append(preds, y_pred)


In [ ]:
# 1
preds = np.array([])
best_params = {}
best_values = {}  # New dictionary to store the best values
with open('best_params_values_0825_6.txt', 'w') as f:
    # Select the data for the current num
    num_data = train.copy()

    # Combine 'hour', 'day', 'week' to a new feature as stratified target
    #num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
    num_data['stratified_target'] = num_data['건물번호'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['hour'].astype(str)

    # Split the data into training and validation set
    train_df, val_df = train_test_split(num_data, test_size=0.7, stratify=num_data['stratified_target'], random_state=724)

    # Drop the temporary feature
    train_df = train_df.drop(columns='stratified_target')
    val_df = val_df.drop(columns='stratified_target')

    # Validation set에서 day 값이 0이 아닌 행을 training set에 추가
    train_df = pd.concat([train_df, val_df[~val_df['day'].isin([6])]])

    # Validation set에서 day 값이 0인 행만 남기기
    val_df = val_df[val_df['day'].isin([6])]

    y_train = train_df['전력소비량(kWh)']
    y_val = val_df['전력소비량(kWh)']

    x_train, x_test = train_df.copy(), test.copy()
    x_val = val_df.copy()

    x_train.drop(['전력소비량(kWh)'],axis=1,inplace=True)
    x_val.drop(['전력소비량(kWh)'],axis=1,inplace=True)

    x_test = x_test[x_train.columns]

    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=15)
    param = study.best_trial.params
    best_params = param
    best_values = study.best_trial.value  # Store the best value
    f.write(f'Best Params: {best_params}, \nBest Values: {best_values}\n\n')
    f.flush()
    xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=953)
    alpha_value2 = xgb.get_params()['alpha']
    xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    pred_6 = np.append(preds, y_pred)


In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/electric/sample_submission.csv')
submission['answer_0'] = pred_0
submission['answer_1'] = pred_1
submission['answer_2'] = pred_2
submission['answer_3'] = pred_3
submission['answer_4'] = pred_4
submission['answer_5'] = pred_5
submission['answer_6'] = pred_6

# 'num_date_time'에서 년월일 부분만 추출
submission['date'] = submission['num_date_time'].apply(lambda x: x.split('_')[1])
submission['date'] = submission['date'].apply(lambda x: x.split(' ')[0])

# 먼저 'answer' 컬럼을 'answer_34'와 'answer_5'의 평균으로 초기화
submission['answer'] = (submission['answer_0'] + submission['answer_1'] + submission['answer_2'] + submission['answer_3'] + submission['answer_4'] + submission['answer_5'] + submission['answer_6']) / 7

print(submission)
# 'date'가 '20220826' 또는 '20220827'인 행의 'answer' 값을 해당 날짜의 'answer_34' 값으로 변경
submission.loc[submission['date'].isin(['20220825', '20220826']), 'answer'] = submission.loc[submission['date'].isin(['20220825', '20220826']), 'answer_34']

# 'date'가 '20220828'인 행의 'answer' 값을 해당 날짜의 'answer_5' 값으로 변경

submission.loc[submission['date'] == '20220825', 'answer'] = submission.loc[submission['date'] == '20220825', 'answer_3']
submission.loc[submission['date'] == '20220826', 'answer'] = submission.loc[submission['date'] == '20220826', 'answer_4']
submission.loc[submission['date'] == '20220827', 'answer'] = submission.loc[submission['date'] == '20220827', 'answer_5']
submission.loc[submission['date'] == '20220828', 'answer'] = submission.loc[submission['date'] == '20220828', 'answer_6']
submission.loc[submission['date'] == '20220829', 'answer'] = submission.loc[submission['date'] == '20220829', 'answer_0']
submission.loc[submission['date'] == '20220830', 'answer'] = submission.loc[submission['date'] == '20220830', 'answer_1']
submission.loc[submission['date'] == '20220831', 'answer'] = submission.loc[submission['date'] == '20220831', 'answer_2']

# 불필요한 컬럼 삭제
submission = submission.drop(columns=['date', 'answer_0', 'answer_1', 'answer_2', 'answer_3', 'answer_4', 'answer_5','answer_6'])

submission.to_csv('7day_optuna_iter15_seed724.csv', index=False)
submission

# ens